<a href="https://colab.research.google.com/github/sho-portfolio/MachineLearning-MultiClassClassifier/blob/master/GoogleColab/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
author:       sho 
date:         2019-11-01
medium:       ()
github:       (https://github.com/sho-portfolio/MachineLearning-MultiClassClassifier)
donate:       ()
description:  code to train, evaluate, productionize and use a model to predict in ~35 lines of code
              four main sections:
              [1] prepare the data
              [2] train and evaluate the model
              [3] productionize the model (save the model and associated files)
              [4] use the saved model to make predictions
"""

In [6]:
#download data source then use it (get single file)
!wget "https://raw.githubusercontent.com/sho-portfolio/MachineLearning-MultiClassClassifier/master/helper.py"

urlTrain = "https://raw.githubusercontent.com/sho-portfolio/MachineLearning-MultiClassClassifier/master/dataTrain.txt"
urlTest = "https://raw.githubusercontent.com/sho-portfolio/MachineLearning-MultiClassClassifier/master/dataTest.txt"
#urlTest = "https://raw.githubusercontent.com/sho-portfolio/MachineLearning-MultiClassClassifier/master/data/simple/dataTest.txt"


--2019-11-04 20:54:52--  https://raw.githubusercontent.com/sho-portfolio/MachineLearning-MultiClassClassifier/master/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3357 (3.3K) [text/plain]
Saving to: ‘helper.py.2’

helper.py.2         100%[===================>]   3.28K  --.-KB/s    in 0s      

2019-11-04 20:54:52 (67.0 MB/s) - ‘helper.py.2’ saved [3357/3357]



In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import sequence
from keras.models import load_model
from keras import metrics
from keras import utils
from subprocess import check_output
import pickle
import helper as helper #only used if dataset is unbalanced or not optimized for ML

In [8]:
## get and prepare data

# load the dataset
df = pd.read_csv(urlTrain)
#df = pd.read_csv('data/dataTrainSO.csv') #stack-overflow dataset
#X_train, y_train, X_test, y_test = helper.load_dataset('dataTrain.txt', 'text', 'label', 0.8)
print(df)

texts = df['text']
labels = df['label']
#texts = df['text'] #use this for the #stack-overflow dataset
#labels = df['label'] #use this for the #stack-overflow dataset

                  text label
0  containsA letterA A     A
1  containsB letterB B     B
2  containsC letterC C     C
3  containsA letterA A     A
4  containsB letterB B     B
5  containsC letterC C     C


In [0]:
# Split data into train and test (80% used for training, 20% for validation) (X = text Y = label)
train_size = int(len(df) * 0.8)
X_train = texts[:train_size]
y_train = labels[:train_size]
X_test = texts[train_size:]
y_test = labels[train_size:]

In [0]:
# prepare (encode) model input data (using tokenizer)
tok = Tokenizer()
tok.fit_on_texts(X_train)
X_train_enc = tok.texts_to_matrix(X_train, mode='count')
X_test_enc = tok.texts_to_matrix(X_test, mode='count')

In [0]:
# prepare (encode) target data (the labels that should be predicted)
num_classes = y_train.nunique()
le = LabelEncoder()
le.fit_transform(y_train)
y_train_enc = le.transform(y_train)
y_test_enc = le.transform(y_test) #test labels transformed but not fit
y_train_enc_categorical = utils.to_categorical(y_train_enc, num_classes)
y_test_enc_categorical = utils.to_categorical(y_test_enc, num_classes)

In [0]:
#print("\n X_train_enc: \n", X_train_enc)
#print("\n X_test_enc: \n", X_test_enc)
#print("\n y_train_enc: \n", y_train_enc)
#print("\n y_test_enc: \n", y_test_enc)
#print("\n y_train_enc_categorical: \n", y_train_enc_categorical)
#print("\n y_test_enc_categorical: \n", y_test_enc_categorical)

In [0]:
#specify the model parameters
epochs=40
number_of_neurons = 500
batch_size = 128
dropout_fraction = 0.5
vocab_size = min(len(tok.word_index) + 1, 1000)

In [16]:
#define the model
model = Sequential()
model.add(Dense(number_of_neurons, input_shape=(vocab_size,), activation='relu'))
model.add(Dropout(dropout_fraction, seed=None))
model.add(Dense(num_classes)) 
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.categorical_accuracy])
print(model.summary())




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               5500      
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 1503      
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total params: 7,003
Trainable params: 7,003
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
# fit the model
model.fit(X_train_enc, y_train_enc_categorical, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 3 samples, validate on 1 samples
Epoch 1/40





3/3 [==============================] - 1s 313ms/step - loss: 1.0497 - categorical_accuracy: 0.3333 - val_loss: 0.8872 - val_categorical_accuracy: 1.0000
Epoch 2/40
3/3 [==============================] - 0s 2ms/step - loss: 1.0661 - categorical_accuracy: 0.3333 - val_loss: 0.8626 - val_categorical_accuracy: 1.0000
Epoch 3/40
3/3 [==============================] - 0s 3ms/step - loss: 0.9253 - categorical_accuracy: 1.0000 - val_loss: 0.8362 - val_categorical_accuracy: 1.0000
Epoch 4/40
3/3 [==============================] - 0s 1ms/step - loss: 1.0407 - categorical_accuracy: 0.6667 - val_loss: 0.8090 - val_categorical_accuracy: 1.0000
Epoch 5/40
3/3 [==============================] - 0s 1ms/step - loss: 0.8774 - categorical_accuracy: 1.0000 - val_loss: 0.7813 - val_categorical_accuracy: 1.0000
Epoch 6/40
3/3 [============================

In [18]:

# evaluate the accuracy of the trained model
score = model.evaluate(X_test_enc, y_test_enc_categorical, batch_size=batch_size, verbose=1)
print("score: ", score)

2/2 [==============================] - 0s 973us/step
score:  [0.20228077471256256, 1.0]


In [0]:
## Productionize the model

#save the model (so that it can be used later)
model.save('model.h5')

#save/Pickle the tokenizer
with open('tokenizer.pickle', 'wb') as handle: pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

#save/Pickle the Label Encoder
with open('labelencoder.pickle', 'wb') as handle: pickle.dump(le, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:

## Use the saved model to make predictions

In [0]:
#load the model
model = load_model('model.h5')

#load the label encoder
with open('labelencoder.pickle', 'rb') as handle: le = pickle.load(handle)

# load the tokenizer
with open('tokenizer.pickle', 'rb') as handle: tok = pickle.load(handle)

In [0]:
#create a dataframe from the test datafile
df = pd.read_csv(urlTest)
#df = pd.read_csv('data/dataTestSO.txt') #stack-overflow dataset

#encode the text values we want to predict on using the loaded tokenizer pickle file
X_test = df['text']
X_test_enc = tok.texts_to_matrix(X_test, mode='count')

In [23]:
#predict values for the test data
#create the output dataframe
df_output = pd.DataFrame(columns=['text', 'label'])
text_labels = le.classes_

for i in range(0,X_test_enc.shape[0]):
  prediction = model.predict(np.array([X_test_enc[i]]))
  predicted_label = text_labels[np.argmax(prediction)]
  #print("input: ", X_test[i])
  #print("prediction: ", prediction)
  #print("predicted_label: ", predicted_label)
  df_output = df_output.append({'text': X_test.iloc[i], 'label':predicted_label}, ignore_index=True)

print (df_output)

         text label
0   containsA     A
1   containsC     C
2   containsB     B
3   containsB     B
4   containsA     A
5           A     A
6           B     B
7     letterA     A
8         cat     B
9         dog     B
10       insA     B
11       insB     B
